In [1]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


In [2]:
class EfficientNetBMI(nn.Module):
    def __init__(self):
        super().__init__()
        weights = EfficientNet_V2_S_Weights.DEFAULT
        self.backbone = efficientnet_v2_s(weights=weights)
        num_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Identity()

        self.fc = nn.Sequential(
            nn.BatchNorm1d(num_features + 1),
            nn.Linear(num_features + 1, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x_img, x_sex):
        x = self.backbone(x_img)
        x = torch.cat((x, x_sex.unsqueeze(1)), dim=1)
        return self.fc(x).squeeze(1)


In [3]:
class BMIDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.valid_indices = [
            idx for idx in range(len(self.data))
            if os.path.exists(os.path.join(self.img_dir, self.data.loc[idx, "name"].strip()))
        ]

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        row = self.data.loc[real_idx]
        img_path = os.path.join(self.img_dir, row["name"].strip())
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        bmi_z = row["bmi_z"]
        sex = row["sex"]
        return image, torch.tensor(bmi_z, dtype=torch.float32), torch.tensor(sex, dtype=torch.float32)


In [16]:
# Load and prepare test data
test_df = pd.read_csv("/Users/yuhsuanko/Desktop/UChicago/UChicago_Q3/ML_II/Final_Project/BMI/Data/data.csv")
test_df = test_df[test_df["is_training"] == 0].reset_index(drop=True)
if "sex" not in test_df.columns and "gender" in test_df.columns:
    test_df["sex"] = test_df["gender"].map({"Male": 0, "Female": 1})
mean_bmi = 32.53
std_bmi = 8.04
test_df["bmi_z"] = (test_df["bmi"] - mean_bmi) / std_bmi

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_dataset = BMIDataset(test_df, img_dir="/Users/yuhsuanko/Desktop/UChicago/UChicago_Q3/ML_II/Final_Project/BMI/Data/Images", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32)


In [18]:
# Load model and evaluate
model = EfficientNetBMI()
model.load_state_dict(torch.load("/Users/yuhsuanko/Desktop/UChicago/UChicago_Q3/ML_II/Final_Project/best_bmi_model.pt", map_location="cpu"))
model.eval()

all_preds, all_targets = [], []

with torch.no_grad():
    for images, targets, sexes in test_loader:
        outputs = model(images, sexes)
        preds = outputs.numpy()
        true = targets.numpy()
        all_preds.extend(preds)
        all_targets.extend(true)

# Inverse transform predictions
preds_bmi = np.array(all_preds) * std_bmi + mean_bmi
targets_bmi = np.array(all_targets) * std_bmi + mean_bmi

In [19]:
# Compute metrics
mae = mean_absolute_error(targets_bmi, preds_bmi)
rmse = mean_squared_error(targets_bmi, preds_bmi) ** 0.5
r2 = r2_score(targets_bmi, preds_bmi)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")


MAE: 5.57
RMSE: 7.77
R²: 0.2892
